In [59]:
import matplotlib.pyplot as plt
import numpy as np
import pandas 

import mne
from mne import Epochs, compute_covariance, find_events, make_ad_hoc_cov
from mne.datasets import sample
from mne.preprocessing import annotate_movement, compute_average_dev_head_t
from mne.datasets.brainstorm import bst_auditory
from mne.io import read_raw_ctf
from mne.preprocessing import annotate_muscle_zscore
from mne.simulation import (
    add_ecg,
    add_eog,
    add_noise,
    simulate_raw,
    simulate_sparse_stc,
)

print(__doc__)

data_path = sample.data_path()
meg_path = data_path / "MEG" / "sample"
raw_fname = meg_path / "sample_audvis_raw.fif"
fwd_fname = meg_path / "sample_audvis-meg-eeg-oct-6-fwd.fif"

# Load real data as the template
RawEEGDataFile = mne.io.read_raw_fif(raw_fname, preload=True);
RawEEGDataFile.set_eeg_reference(projection=True);
RawEEGDataFile = RawEEGDataFile.pick(picks=["eeg", "eog", "ecg", "stim"]).load_data();
RawEEGDataFile.interpolate_bads();
RawEEGDataFile.get_data().shape
# ChannelNum = input("Please input the number of channels:")
#Freq = float(input("Please input sample refresh rate in Hz:"))
# RawEEGDataFile = input("Please input the EEG data file name:")

Freq = 600
EEGChannelNum = 60

BPEEGDataFile = RawEEGDataFile.copy().filter(l_freq=0.5, h_freq=40.0, fir_design='firwin')
#RawEEGDataFile.notch_filter([60, 120])

#BPEEGDataFile.plot()
#RawEEGDataFile.plot()
df = RawEEGDataFile.to_data_frame()
display(df)


AlphaComp = RawEEGDataFile.compute_psd(method='welch', fmin=8, fmax=12, tmin=None, tmax=None, picks='eeg', exclude=(), proj=False, remove_dc=True, reject_by_annotation=True, n_jobs=1, verbose=None)
AlphaPSD, AlphaFreq = AlphaComp.get_data(return_freqs=True)
display(AlphaComp)
DeltaComp = RawEEGDataFile.compute_psd(method='welch', fmin=0.5, fmax=4, tmin=None, tmax=None, picks='eeg', exclude=(), proj=False, remove_dc=True, reject_by_annotation=True, n_jobs=1, verbose=None)
DeltaPSD, DeltaFreq = DeltaComp.get_data(return_freqs=True)
#DeltaComp.plot()
raw_csd = mne.preprocessing.compute_current_source_density(RawEEGDataFile);

AlphaMean = AlphaPSD.mean(axis=1)
DeltaMean = DeltaPSD.mean(axis=1)


AlDeRat = AlphaMean / DeltaMean

display(AlphaMean.shape)
display(DeltaMean.shape)
display(AlDeRat.shape)


ChannelNum = RawEEGDataFile.ch_names

PSDRatDF = pandas.DataFrame({'Channel': EEGChannelNum,'Alpha Power': AlphaMean,'Delta Power': DeltaMean,'Alpha/Delta Ratio': AlDeRat})
display(AlDeRat)
display(PSDRatDF)



display(RawEEGDataFile)
#AlphaComp[9,1,:]
#np.average(AlphaComp[i, j, :])

#AlpDelRat = AlphaComp / DeltaComp
#print(AlpDelRat)

#raw_csd.plot();


#raw.compute_psd().plot(picks="data", exclude="bads", amplitude=False);

#raw_csd.compute_psd().plot(picks="data", exclude="bads", amplitude=False);


Automatically created module for IPython interactive environment
Opening raw data file C:\Users\maxeg\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...


    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 91.0 mm
Computing interpolation matrix from 59 sensor positions
Interpolating 1 sensors
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stop

,time,STI 001,STI 002,STI 003,STI 004,STI 005,STI 006,STI 014,STI 015,STI 016,...,EEG 052,EEG 053,EEG 054,EEG 055,EEG 056,EEG 057,EEG 058,EEG 059,EEG 060,EOG 061
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.057099,41.936051,43.392211,59.672853,28.921204,50.738707,69.109191,38.854217,65.839113,285.661012
1,0.001665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,43.450562,43.043072,44.788207,61.445314,31.544983,52.276244,70.878295,40.751037,68.002565,283.699953
2,0.003330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,43.757416,42.266710,43.973876,61.208986,31.902772,51.706786,70.650024,40.995788,68.177980,280.431520
3,0.004995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,43.511933,43.057234,44.730041,61.090822,31.664246,52.219298,71.277770,41.179352,68.587282,279.124147
4,0.006660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.977662,43.041405,44.264709,60.145509,30.292725,51.763731,70.364684,39.343719,67.242433,281.738893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166795,277.707021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,64.378022,72.094857,69.974303,84.723635,57.902040,75.453184,96.558837,67.551269,85.427126,256.245119
166796,277.708686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,62.843751,72.464609,70.614134,84.664553,56.828675,75.510130,96.216430,67.551269,85.076296,262.128297
166797,277.710351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,61.554963,72.322715,69.799803,83.719240,55.695680,75.396238,95.874022,67.245330,85.193239,264.089357
166798,277.712016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,61.248109,71.867912,68.985472,82.951174,55.278260,75.453184,95.988158,66.878204,85.894899,262.781984


Effective window size : 3.410 (s)


Data type,Power Spectrum
Units,eeg: V²/Hz
Data source,Raw
Dims,"channel, freq"
Estimation method,welch
Number of channels,60
Number of frequency bins,13
Frequency range,8.21 – 11.73 Hz


...

(60,)

(60,)

array([0.01477769, 0.01599957, 0.01570311, 0.02376167, 0.03133989,
       0.02893706, 0.01608822, 0.03840429, 0.05228898, 0.05500388,
       0.06300029, 0.06301361, 0.0577703 , 0.05900012, 0.03731939,
       0.03207104, 0.07443221, 0.07490601, 0.08262503, 0.09646239,
       0.08559399, 0.06863768, 0.06392195, 0.06977955, 0.10819079,
       0.13165016, 0.14668427, 0.17665733, 0.15769029, 0.15341756,
       0.14762926, 0.15727815, 0.13225163, 0.11041289, 0.11703198,
       0.17594204, 0.20668588, 0.23389586, 0.26215689, 0.23826114,
       0.22356267, 0.16823527, 0.18742366, 0.3479833 , 0.33753202,
       0.30874903, 0.33619193, 0.32910757, 0.31180749, 0.2895753 ,
       0.26223786, 0.25155862, 0.36131377, 0.44396641, 0.36719707,
       0.37027739, 0.39816607, 0.33941206, 0.3223864 , 0.22428811])

,Channel,Alpha Power,Delta Power,Alpha/Delta Ratio
0,60,1.560975e-12,1.056306e-10,0.014778
1,60,1.717624e-12,1.073544e-10,0.016000
2,60,1.775302e-12,1.130542e-10,0.015703
3,60,1.472931e-12,6.198771e-11,0.023762
4,60,1.893365e-12,6.041390e-11,0.031340
5,60,2.072229e-12,7.161162e-11,0.028937
6,60,1.658087e-12,1.030622e-10,0.016088
7,60,1.479492e-12,3.852413e-11,0.038404
8,60,1.871563e-12,3.579269e-11,0.052289
9,60,2.335630e-12,4.246300e-11,0.055004


<Raw | sample_audvis_raw.fif, 70 x 166800 (277.7 s), ~92.0 MB, data loaded>

In [ ]:
all_events, all_event_id = mne.events_from_annotations(raw)
mne.viz.plot_events(events=all_events, event_id=all_event_id, sfreq=raw.info["sfreq"])

In [ ]:
for a, annotation in enumerate(annot_muscle):
        if annotation['duration']<0.3:
            remove.append(a)
            
    annot_muscle.delete(remove)


In [1]:
import mne
from mne.viz import plot_alignment, set_3d_view

print(__doc__)

data_path = mne.datasets.sample.data_path()
subjects_dir = data_path / 'subjects'
meg_path = data_path / 'MEG' / 'sample'
trans = mne.read_trans(meg_path / 'sample_audvis_raw-trans.fif')
raw = mne.io.read_raw_fif(meg_path / 'sample_audvis_raw.fif')

# Plot electrode locations on scalp
fig = plot_alignment(raw.info, trans, subject='sample', dig=False,
                     eeg=['original', 'projected'], meg=[],
                     coord_frame='head', subjects_dir=subjects_dir)

# Set viewing angle
set_3d_view(figure=fig, azimuth=135, elevation=80)

Automatically created module for IPython interactive environment
Opening raw data file C:\Users\maxeg\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.


RuntimeError: Could not load any valid 3D backend
pyvistaqt: No module named 'qtpy'
notebook: No module named 'ipyevents'

 install pyvistaqt, using pip or conda:
'pip install pyvistaqt'
'conda install -c conda-forge pyvistaqt'

 or install ipywidgets, if using a notebook backend
'pip install ipywidgets'
'conda install -c conda-forge ipywidgets'